In [2]:
import glob
import model
import data_process
from keras.callbacks import CSVLogger
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler

In [3]:
def step_decay(epoch):
    initial_lrate = 0.001
    drop = 0.4
    epochs_drop = 15.0
    lrate = initial_lrate * math.pow(drop,  
            math.floor((1+epoch)/epochs_drop))
    
    if (lrate < 4e-5):
        lrate = 4e-5
      
    print('Changing learning rate to {}'.format(lrate))
    return lrate

In [4]:
csv_logger = CSVLogger("../check_point/model-attRNN.csv", append=True)

numClass = 15

In [ ]:
trainData, valData, testData = data_process.load_data()

trainGen = data_process.SpeechGen(trainData)
valGen = data_process.SpeechGen(valData)

In [ ]:
model = model.AttRNNSpeechModel(numClass, samplingrate = 8000, inputLength = 16000)

model.compile(optimizer='adam', loss=['sparse_categorical_crossentropy'], metrics=['sparse_categorical_accuracy'])

model.summary()

In [ ]:
lrate = LearningRateScheduler(step_decay)

earlystopper = EarlyStopping(monitor='val_sparse_categorical_accuracy', patience=10,
                             verbose=1, restore_best_weights=True)

checkpointer = ModelCheckpoint('../check_point/model-attRNN.hdf5', monitor='val_sparse_categorical_accuracy', verbose=1, save_best_only=True)

results = model.fit(trainGen, validation_data=valGen, epochs=60, verbose=1,
                    callbacks=[earlystopper, checkpointer, lrate, csv_logger])

In [ ]:
# summarize history for categorical accuracy
plt.plot(results.history['sparse_categorical_accuracy'])
plt.plot(results.history['val_sparse_categorical_accuracy'])
plt.title('Categorical accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# summarize history for loss
plt.plot(results.history['loss'])
plt.plot(results.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
testGen = data_process.SpeechGen(testData, batch_size=len(testData), shuffle=False)
x_test, y_test = testGen.__getitem__(0)

In [ ]:
valEval = model.evaluate(valGen, use_multiprocessing=False, workers=4,verbose=0)
trainEval = model.evaluate(trainGen, use_multiprocessing=False, workers=4,verbose=0)
testEval = model.evaluate(x_test, y_test, verbose=0)
print('Evaluation scores: \nMetrics: {} \nTrain: {} \nValidation: {} \nTest: {}'.format(model.metrics_names, trainEval, valEval, testEval) )

In [ ]:
y_pred = model.predict(x_test, verbose=1)

In [ ]:
from sklearn.metrics import confusion_matrix
import audioUtils
cm = confusion_matrix(y_test, np.argmax(y_pred,1))

In [ ]:
className = ['Close_door', 'Close_gate', 'Doraemon', 'Lock_door', 'Lock_gate', 'Open_door', 'Open_gate', 'Turn_off_air_conditioner', 'Turn_off_fan', 'Turn_off_light', 'Turn_off_TV', 'Turn_on_air_conditioner', 'Turn_on_fan', 'Turn_on_light', 'Turn_on_TV']

In [ ]:
audioUtils.plot_confusion_matrix(cm, classes, 'confusion.png', normalize=False)